In [6]:
import os
import json
import pandas as pd
import traceback


In [2]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import openai
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key = api_key, model_name = "GPT-4.1 nano, temperature = 0.5")
# higher the value temperature, more creative is the model

C:\Users\aakri\AppData\Local\Temp\ipykernel_7536\4272986038.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key = api_key, model_name = "GPT-4.1 nano, temperature = 0.5")
